## Descripción del escenario (Jupyter-Cassandra)

Este ejemplo muestra la interconexión de Jupyter con Cassandra. El principal objetivo es la descarga de dos datasets desde internet y luego guardar los datos en Cassandra. 

* Obtención de dos datasets desde internet
* Guardado de los datos en Cassandra

###  Cargar las librerias al Notebook 

    Pandas : Libreria destinada al análisis de datos proporcionando estructura de datos flexibles y la 
    utilizamos para la carga de ficheros csv. 
    
    Cassandra: Libreria para hacer la conexión con la base de datos de Cassandra. 
    

In [1]:
import pandas as pd
import io
import time
import requests
from cassandra.cluster import Cluster
from cassandra.query import *

/home/legacy/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """


### Cargamos Primer fichero CSV desde URL

* Movies.csv:

Se realiza la descarga de un fichero movies.csv almacenado en un repositorio externo que contiene la información sobre un conjunto de películas 

El fichero tiene los siguientes campos: movieID, Title, Genres. 




In [2]:
url = "https://dbox.iti.upv.es/oc/index.php/s/2kUw26FLKEBWAZy/download"
s = requests.get(url).content

# Se realiza la lectura del fichero y se almacena en dfMovies. 

dfMovies = pd.read_csv(io.StringIO(s.decode('utf-8')))

# Imprimimos el contenido del fichero que contiene aproximadamente 45843 líneas

dfMovies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


### Cargamos Segundo fichero CSV desde URL

* rating.csv:

Se realiza la descarga de un fichero rating.csv almacenado en un repositorio externo que contiene la información rating de las películas 

El fichero tiene los siguientes campos: userid, movieId, ratinf, timestamp

In [3]:
# A continuación se tienen tres tamaños para el fichero rating.csv. Por defecto hemos seleccionado un fichero con 
# 100000 registros. 

url = "https://dbox.iti.upv.es/oc/index.php/s/nabnh61C5yQEFlW/download"  # 100000 registros
#url = "https://dbox.iti.upv.es/oc/index.php/s/xlQ1u8Y2a6s5Kd2/download"  # 200000 registros
#url = "https://dbox.iti.upv.es/oc/index.php/s/EnBCop2gm4y1tCz/download"  # 400000 registros

s = requests.get(url).content

# Se realiza la lectura del fichero y se almacena en dfRatings.

dfRatings = pd.read_csv(io.StringIO(s.decode('utf-8')))

# Imprimimos el contenido del fichero que contiene los ratings. 

dfRatings

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
5,1,1968,4.0,1425942148
6,1,2762,4.5,1425941300
7,1,2918,5.0,1425941593
8,1,2959,4.0,1425941601
9,1,4226,4.0,1425942228


### Creamos Keyspace y tablas en Cassandra

Creación del espacio de claves **test** en Cassandra y las tablas **movies** y **ratings** para insertar datos.

Si las tablas existen en Cassandra las borra previamente

In [5]:
# Hace lo conexión con la base de datos de Cassandra. 

cluster = Cluster(['172.19.0.2'], control_connection_timeout = 50, connect_timeout = 50)
session = cluster.connect()

cmds = [
    #"DROP KEYSPACE IF EXISTS test",
    "CREATE KEYSPACE IF NOT EXISTS test WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 2}",
    "USE test",
    "DROP TABLE IF EXISTS movies",
    "DROP TABLE IF EXISTS ratings",
    "CREATE TABLE IF NOT EXISTS movies (movie_id int primary key, title text, genres text)",
    "CREATE TABLE IF NOT EXISTS ratings (user_id int, movie_id int, rating float, timestamp bigint, PRIMARY KEY (user_id, movie_id))"
]

for cmd in cmds:
    print(cmd)
    session.execute(cmd, timeout = 50)

NameError: name 'seed' is not defined

### Inserción de datos en Cassandra

Inserción de los datos en las tablas movies y ratings. La inserción se hace en batches de 100 registros

In [ ]:
inicio = time.time()

# Preparar un esqueleto para la inserción de registros en Cassandra.

insert = session.prepare("INSERT INTO movies (movie_id, title, genres) VALUES (?, ?, ?)")

# Calculamos el número de Iteraciones para guardar el fichero de 100 en 100 registros. 

moviesIter= int(len(dfMovies)/100)
if (len(dfMovies)%100!=0):
    moviesIter+=1
print(moviesIter)

for bn in range(moviesIter):
    batch = BatchStatement(consistency_level = ConsistencyLevel.ONE)
    for r in dfMovies[bn*100 : bn*100 + 100].itertuples():
        batch.add(insert, (r.movieId, r.title, r.genres))
    session.execute(batch)
fin = time.time()

print("Tiempo: " + str((fin - inicio)))

In [ ]:
inicio = time.time()

# Preparar un esqueleto para la inserción de registros en Cassandra para la tabla movie.

insert = session.prepare("INSERT INTO ratings (user_id, movie_id, rating, timestamp) VALUES (?, ?, ?, ?)")

for bn in range(int(len(dfRatings.index) / 100)):
    batch = BatchStatement(consistency_level = ConsistencyLevel.ONE)
    for r in dfRatings[bn*100 : bn*100 + 100].itertuples():
        batch.add(insert, (r.userId, r.movieId, r.rating, r.timestamp))
    session.execute(batch)
fin = time.time()

print("Tiempo: " + str((fin - inicio)))